# Dataset Cleaning and Exploration

To prepare for our ultimate task of using molecular embeddings to predict abundances and detectability, we need to put our dataset into working order. This is typically referred to as "cleaning", where we're making sure that the data will be valid (free of missing data, for example) and duplicate entries are removed. We will also need to inspect the data to make sure that entries we expect to be there are present, as well as observe some general trends where we can. 

Since we're looking at such a large dataset, we need to be able to inspect it from a microscopic and a macroscopic level. Combining both perspectives gives you an overview of what the dataset looks like, and in turn may give you insight into why/how a machine learning model behaves the way it does.

The first part of this notebook will be combining the three datasets: QM9, ZINC15, and KIDA. The latter is actually obtained by scraping their website, i.e. extracting the relevant information from tables in websites. 

In [119]:
from pathlib import Path
from tempfile import NamedTemporaryFile
import fileinput
import os

import numpy as np
import pandas as pd
from mol2vec import features
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from gensim.models import word2vec

## Loading and combining SMILES from the two datasets

In [96]:
qm9 = pd.read_csv("../data/gdb9_prop_smiles.csv.tar.gz")
smi_list = qm9["smiles"].dropna().to_list()

In [97]:
for smi_file in Path("../data/").rglob("*/*.smi"):
    temp = smi_file.read_text().split("\n")[1:]
    for line in temp:
        if len(line) != 0:
            smi_list.append(line.split(" ")[0])

In [98]:
print(f"Number of molecules: {len(smi_list)}")

Number of molecules: 1543543


## Extracting SMILES from KIDA

Since our database contains mostly terrestrial/stable molecules, we need to augment this set with astronomically relevant molecules. KIDA is one of the biggest reaction networks used in astrochemistry, and is therefore a nice collection molecules that may or may not be found in space (at least of interest).

To perform this, we'll scrape the KIDA website below.

In [66]:
import requests
import datetime
from bs4 import BeautifulSoup

In [25]:
url = requests.get("http://kida.astrophy.u-bordeaux.fr/species.html")

In [30]:
print(f"""Last retrieved: {url.headers["Date"]}""")

Last retrieved: Sun, 05 Jul 2020 20:26:13 GMT


In [101]:
date = url.headers["Date"]
# cut the date, replace spaces with underscores for naming
date = date[5:16].replace(" ", "_")
# save the webpage for reference. If KIDA decides to go bottom up we
# will always have a copy of this data
with open(f"../data/kida-site_{date}.html", "w+") as write_file:
    write_file.write(str(url.content))

In [33]:
soup = BeautifulSoup(url.content)

In [37]:
# the first and only table on the page corresponds to the molecules
molecule_table = soup.find_all("table", "table")[0]

In [54]:
map_dict = dict()
for row in soup.find_all("tr"):
    # some InChI are missing, this sets a default value
    inchi = None
    for index, column in enumerate(row.find_all("td")):
        # loop over columns in each row, and grab the second and
        # third columns which are formulae and InChI
        if index == 1:
            # strip twice because the first header is parsed funnily
            name = column.text.strip().strip()
        if index == 2:
            inchi = column.text.strip()
    map_dict[name] = inchi

In [63]:
# Just for reference, dump the KIDA mapping as a dataframe
kida_df = pd.DataFrame.from_dict(map_dict, orient="index").reset_index()
kida_df.columns = ["Formula", "InChI"]

In [76]:
kida_df.to_csv(f"../data/kida-molecules_{date}.csv", index=False)

Now we convert all the InChI codes from KIDA into SMILES through RDKit. Initially I was most worried about this because KIDA has strange molecules, and as we see below RDKit has plenty to complain about. The attitude we're taking here is to ignore the ones that don't play by the rules, and we'll worry about them some other time.

In [93]:
ignored = list()
kida_smiles = dict()
for key, value in map_dict.items():
    # ignore missing InChI
    if len(value) != 0:
        mol = Chem.MolFromInchi(value, sanitize=False, removeHs=False)
        if mol:
            kida_smiles[key] = Chem.MolToSmiles(mol, canonical=True)
    else:
        ignored.append(key)

RDKit WARNING: [16:46:31] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:31] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:31] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:31] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:35] expect radical to be either 2 or 3 while getting . Ignore radical.
RDKit WARNING: [16:46:35] expect radical to be either 2 or 3 while getting . Ignore radical.
RDKit WARNING: [16:46:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [16:46:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s)

In [102]:
# append all the KIDA entries to our full list
smi_list.extend(list(kida_smiles.values()))

In [104]:
print(f"Number of molecules with KIDA: {len(smi_list)}")

Number of molecules with KIDA: 1544055


## Microscopic inspection

In this section, we're going to put certain aspects of the dataset under the microscope: for example, we want to check that certain molecules are contained in the set. Here, we'll be using our chemical intuition; the idea is to pick out a few molecules, and check if: (a) they are contained in our list, and (b) what their most similar molecules are.

In [105]:
mol_df = pd.DataFrame(data=smi_list)
mol_df.columns = ["Raw"]

In [106]:
def canonicize_smi(smi: str):
    """
    Function to convert any SMILES string into its canonical counterpart.
    This ensures that all comparisons made subsequently are made with the
    same SMILES representation, if it exists.
    """
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi, sanitize=False), canonical=True)

In [112]:
checklist = [
    "CC=O",             # acetaldehyde
    "c1ccccc1",         # benzene
    "c1ccc(cc1)C#N",    # benzonitrile
    "N#CC=C",           # vinyl cyanide
    "CC#N",             # methyl cyanide
    "C#CC#CC#N",
    "C#N",
    "C#CC#CC#CC#N",
    "C#CC#CC#CC#CC#N",
]

checklist = [canonicize_smi(smi) for smi in checklist]

In [108]:
mol_df.loc[:, "Check"] = mol_df["Raw"].isin(checklist)

In [109]:
mol_df.loc[mol_df["Check"] == True]

,Raw,Check
4,C#N,True
9,CC#N,True
10,CC=O,True
484,C#CC#CC#N,True
14562,C#CC#CC#CC#N,True
571628,N#Cc1ccccc1,True
585302,c1ccccc1,True
1543684,C=CC#N,True
1543920,C#CC#CC#CC#CC#N,True


In [126]:
molecular_weights = list()
for smi in smi_list:
    mol = Chem.MolFromSmiles(smi, sanitize=False)
    mol.UpdatePropertyCache(strict=False)
    molecular_weights.append(Descriptors.ExactMolWt(mol))

## Calculate descriptors

In [128]:
mol_df["MW"] = molecular_weights

## Drop duplicate entries, and save the data to disk

Our dataset actually contains a lot of duplicate entries. This step removes them, which would otherwise just waste our computation time.

In [136]:
mol_df.drop_duplicates("Raw", inplace=True)

In [141]:
mol_df.to_pickle("../data/combined_smiles.pkl.bz2")

## Tasks for data exploration

### Distribution of molecular weight

Plot a histogram of the molecular weight in our dataset.

### Counting functional group examples

In [147]:
# For example, number of carbonyls
mol_df["Raw"].str.contains("C=O").sum()

17782

## Dumping the SMILES data for mol2vec use

This only takes the SMILES column, and dumps it into a list of SMILES formatted and ready for `mol2vec` usage. Every SMILES is separated by a new line, and we don't include a header.

In [154]:
mol_df["Raw"].to_csv("./collected_smiles.smi", sep="\n", index=False, header=None)